## PreTrained model testing

In [ ]:
import os
from urllib.request import urlretrieve
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
url_dog="https://static.posters.cz/image/1300/fototapetak/golden-retriever-dog-416x254-cm-premium-non-woven-130gsm-i81322.jpg"
urlretrieve(url_dog, "dog.jpg")

('dog.jpg', <http.client.HTTPMessage at 0x7fb201a0e7b8>)

In [ ]:
model = InceptionV3(weights='imagenet', include_top=True)

96116736/96112376 [==============================] - 1s 0us/step


In [ ]:
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
preds = model.predict(x)
print('Jósolt osztály:')
print(decode_predictions(preds))

Jósolt osztály:
40960/35363 [==================================] - 0s 0us/step
[[('n02099601', 'golden_retriever', 0.6171925), ('n11879895', 'rapeseed', 0.26706514), ('n02099712', 'Labrador_retriever', 0.052262116), ('n02104029', 'kuvasz', 0.009309961), ('n04409515', 'tennis_ball', 0.0035573721)]]


# PreTrained model - Transfer learning (továbbtanítás)

Képek betöltése

In [ ]:
import os
import zipfile

In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O /tmp/cats_and_dogs_filtered.zip

--2020-11-25 21:18:41--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.240, 172.217.13.240, 172.217.12.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   207MB/s    in 0.3s    

2020-11-25 21:18:42 (207 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered' 
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Pretrained model betöltése, továbbtanítása

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
img_height=299
img_width=299

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='sigmoid')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='SGD', metrics=['accuracy'],loss='binary_crossentropy')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.8, zoom_range=0.8, horizontal_flip=True, rotation_range = 45)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator,steps_per_epoch=50,validation_data=validation_generator,validation_steps=10,epochs=3)

Epoch 1/3
50/50 [==============================] - 26s 518ms/step - loss: 0.5773 - accuracy: 0.7840 - val_loss: 0.4168 - val_accuracy: 0.9700
Epoch 2/3
50/50 [==============================] - 25s 497ms/step - loss: 0.5468 - accuracy: 0.8520 - val_loss: 0.4183 - val_accuracy: 0.9800
Epoch 3/3
50/50 [==============================] - 25s 498ms/step - loss: 0.5235 - accuracy: 0.8640 - val_loss: 0.4128 - val_accuracy: 0.9800


In [ ]:
for layer in model.layers[:172]:
       layer.trainable = False
for layer in model.layers[172:]:
       layer.trainable = True

In [ ]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'], loss='binary_crossentropy')

In [ ]:
model.fit_generator(train_generator,steps_per_epoch=50,validation_data=validation_generator,validation_steps=10,epochs=3)

Epoch 1/3


KeyboardInterrupt: ignored